# Извлечение сообщений

извлекаю посты за 2017-2023 года из определенных групп вк (их id достаю из бд вк в лабе), имеющих отношение к ЖКХ

In [ ]:
# !pip install vk

Note: you may need to restart the kernel to use updated packages.


In [3]:
import getpass
import time
from collections import defaultdict

from datetime import datetime

import vk
from vk.exceptions import VkAPIError

import pandas as pd

from tqdm.notebook import tqdm

# нужно завести аккаунт здесь https://vk.com/dev , потом создать приложение, оттуда достать сервисный ключ и ввести здесь
api = vk.API(access_token=getpass.getpass())

In [2]:
# этот класс нужен, чтобы автоматически процесс засыпал через несколько вызовов подряд,
# чтобы уложиться в лимиты по количеству вызовов
class RequestWithSleep:
    def __init__(self, max_calls_per_second=3, sleep_time=1):
        self.max_calls_per_second = max_calls_per_second
        self.sleep_time = sleep_time
        self.counter = 0
    
    def request(self, method, **kwargs):
        if self.counter % self.max_calls_per_second == 0:
            time.sleep(self.sleep_time)
        self.counter += 1
        return method(**kwargs)

requester = RequestWithSleep()

In [ ]:
# мне нужны только поля ['id', 'owner_id', 'from_id', 'date', 'text']

APIVersion = 5.131

groups_df = pd.read_csv('jkh_groups/jkh_groups3.csv')
# фильтрация групп, чтобы не доставать сообщения из мусорных групп
cond = ~groups_df.name.str.contains(r'отдам|закупки|шугаринг|сервисный|страхование|ремонт|покупки|cake|студия|компьютер|косметолог|шар|услуги|ресниц|маникюр|малыш|oriflame|детский|Дед Мороз|недорого', case=False) & \
    groups_df.name.str.contains('ЖК')
groups_df = groups_df[cond]
groups_ids = -groups_df['id']

group_posts = defaultdict(list)

def extract_data_from_item(item):
    data = {}
    for key in ['id', 'owner_id', 'from_id', 'date', 'text']:
        data[key] = item[key]
    return data

for owner_id in tqdm(groups_ids):
    try:
        data = requester.request(api.wall.get, owner_id=owner_id, offset=0, count=100, v=APIVersion)
        group_posts[owner_id].extend(map(extract_data_from_item, data['items']))

        user_posts_count = min(300, data['count'])
        for i in range(100, user_posts_count, 100):
            data = requester.request(api.wall.get, owner_id=owner_id, offset=i, count=min(user_posts_count - i, 100), v=APIVersion)
            group_posts[owner_id].extend(map(extract_data_from_item, data['items']))

            if datetime.fromtimestamp(group_posts[owner_id][-1]['date']).year < 2017:
                break

            if i % 500 == 0:
                print(i)
    
    except VkAPIError as e:
        if str(e).startswith('6.'): print(e)

  0%|          | 0/4674 [00:00<?, ?it/s]

In [5]:
import json

with open('jkh_groups/posts_part_1.json', 'w') as outfile:
    json.dump(group_posts, outfile)

In [6]:
sum(len(x) for x in group_posts.values())

371311

In [7]:
# сохранение в csv

import os
import pandas as pd

all_data = []
for filename in os.listdir('jkh_groups'):
    if not filename.startswith('posts_'):
        continue
    with open(f'jkh_groups/{filename}', 'r') as f:
        data = json.load(f)
        for lst in data.values():
            all_data.extend(lst)

jkh_posts_data = {}
for key in all_data[0]:
    jkh_posts_data[key] = list(map(lambda x: x[key], all_data))

jkh_posts_df = pd.DataFrame(jkh_posts_data)
jkh_posts_df.to_csv('jkh_groups/jkh_posts3_dirty.csv')